Import Necessary 

In [ ]:
import os
import ultralytics
import supervision as sv
import numpy as np


In [ ]:
%pip install ultralytics
%pip install supervision
%pip install pytube

Video_Sample_Download

In [ ]:
from pytube import YouTube

# Replace 'your_video_url' with the actual YouTube video URL
video_url = 'https://www.youtube.com/watch?v=KFnavYR4g_U'

# Create a YouTube object
yt = YouTube(video_url)

# Get the stream with the desired resolution and file type (e.g., 'mp4')
video_stream = yt.streams.filter(res='720p', file_extension='mp4').first()

# Specify the download location
download_location = 'Vehicle_Tracking/yoloV8Implementation/Testing_Files'

# Download the video
video_stream.download()

print(f"Video '{yt.title}' downloaded successfully in 420p MP4 format!")

In [1]:
sourceVideoPath = "Testing_Files\Testv3.mp4"
TARGET_VIDEO_PATH = f"Result_Files\ vehicle-counting-result-with-counter.mp4"


Load PreTrained Models

In [ ]:
MODEL = "yolov8n.pt"

In [ ]:
from ultralytics import YOLO
model= YOLO(MODEL)
model.fuse()

In [ ]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names

# class_ids of interest - car, motorcycle, bus and truck
selected_classes = [1, 2, 3, 5, 7]
#Selecting model classes for use
#  1: 'bicycle', 2: 'car', 3: 'motorcycle', 5: 'bus', 7: 'truck',


Visualizing

In [ ]:
# create frame generator
generator = sv.get_video_frames_generator(sourceVideoPath)
# create instance of BoxAnnotator
box_annotator = sv.BoxAnnotator(thickness=1, text_thickness=1)
# acquire first video frame
iterator = iter(generator)
frame = next(iterator)
# model prediction on single frame and conversion to supervision Detections
results = model(frame, verbose=False)[0]

# convert to Detections
detections = sv.Detections.from_ultralytics(results)
# only consider class id from selected_classes define above
detections = detections[np.isin(detections.class_id, selected_classes)]

# format custom labels
labels = [
    f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for confidence, class_id in zip(detections.confidence, detections.class_id)
]

# annotate and display frame
anotated_frame=box_annotator.annotate(scene=frame, detections=detections, labels=labels)

%matplotlib inline
sv.plot_image(anotated_frame, (16,16))

Create Lines For Vehicle Counts

In [ ]:
# settings
LINE_START = sv.Point(10, 744)
LINE_END = sv.Point(1910, 744)

In [ ]:
sv.VideoInfo.from_video_path(sourceVideoPath)

Video Track Using ByteTracker

In [ ]:
# create BYTETracker instance
byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30)
# create VideoInfo instance
video_info = sv.VideoInfo.from_video_path(sourceVideoPath)

# create frame generator
generator = sv.get_video_frames_generator(sourceVideoPath)

# create LineZone instance, it is previously called LineCounter class
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)

# create instance of BoxAnnotator
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)

# create instance of TraceAnnotator
trace_annotator = sv.TraceAnnotator(thickness=4, trace_length=50)

# create LineZoneAnnotator instance, it is previously called LineCounterAnnotator class
line_zone_annotator = sv.LineZoneAnnotator(thickness=4, text_thickness=4, text_scale=2)

# define call back function to be used in video processing
def callback(frame: np.ndarray, index:int) -> np.ndarray:
    # model prediction on single frame and conversion to supervision Detections
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    # only consider class id from selected_classes define above
    detections = detections[np.isin(detections.class_id, selected_classes)]
    # tracking detections
    detections = byte_tracker.update_with_detections(detections)
    labels = [
        f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
        for confidence, class_id, tracker_id
        in zip(detections.confidence, detections.class_id, detections.tracker_id)
    ]
    annotated_frame = trace_annotator.annotate(
        scene=frame.copy(),
        detections=detections
    )
    annotated_frame=box_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        labels=labels)

    # update line counter
    line_zone.trigger(detections)
    # return frame with box and line annotated result
    return  line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

# process the whole video
sv.process_video(
    source_path = sourceVideoPath,
    target_path = TARGET_VIDEO_PATH,
    callback=callback
)

Review

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = 'Vehicle_Tracking\yoloV8Implementation\Result_Files\vehicle-counting-result-with-counter.mp4'

# Compressed video path
compressed_path = "Vehicle_Tracking\yoloV8Implementation\Result_Files\compressed\result_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)